In [20]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.chat_models.openai import ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain.chains.question_answering import load_qa_chain

In [21]:
import os

In [22]:
import dotenv

In [23]:
load_ext = dotenv.load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [24]:
# Load the Models (Embedding and LLM)

embedding_model = OpenAIEmbeddings()
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, max_tokens=200)

In [25]:
# Load PDF document
PDF_NAME = "pdf_example.pdf"
loader = PyPDFLoader(PDF_NAME, extract_images=False)
pages = loader.load_and_split()

In [26]:
# Split the text into chunks
# What is chunk? A chunk is a smaller, manageable piece of text extracted from a larger document.
# What is overlap? Overlap refers to the number of characters that are repeated between consecutive chunks to maintain context.

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=4000,
    chunk_overlap=20,
    length_function=len,
    add_start_index=True
)

chunks = text_splitter.split_documents(pages) # Split the document into chunks

In [27]:
# Save the chunks into a vector store

database = Chroma.from_documents(chunks, embedding=embedding_model, persist_directory="text_index")
database.persist() # Save to disk

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
# Load the vector store from disk
vector_database = Chroma(embedding_function=embedding_model, persist_directory="text_index")

# Load Retriever from the vector store
retriever = vector_database.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# Create the QA Chain
qa_chain = load_qa_chain(llm, chain_type="stuff")

/var/folders/cc/sf3s5jt10s524c_dq5vd5_f40000gn/T/ipykernel_12808/298918895.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_database = Chroma(embedding_function=embedding_model, persist_directory="text_index")
Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
/var/folders/cc/sf3s5jt10s524c_dq5vd5_f40000gn/T/ipykernel_12808/298918895.py:8: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrat

In [ ]:
def answer_question(question: str) -> str:
    # Retrieve relevant documents
    relevant_docs = retriever.get_relevant_documents(question)
    
    # Generate the answer using the QA chain
    answer = qa_chain.run(input_documents=relevant_docs, question=question)
    
    return answer

# Example usage
question = "What is Rocketseat?"
answer = answer_question(question)
print("Question:", question)
print("Answer:", answer)

/var/folders/cc/sf3s5jt10s524c_dq5vd5_f40000gn/T/ipykernel_12808/4242329217.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  relevant_docs = retriever.get_relevant_documents(question)


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}